In [9]:
from importlib import reload

import numpy as np

from ics.cobraControl import ethernet
from ics.cobraControl import func, cmds
from ics.cobraControl import convert


class PFI(object):
    CW = 1
    CCW = -1
    DISABLE = 0
    
    nCobrasPerModule = 57
    nModules = 42
    
    def __init__(self, fpgaHost='localhost', doConnect=True, doLoadModel=True):
        self.fpgaHost = fpgaHost
        if doConnect:
            self.connect()
        if doLoadModel:
            self.loadModel()
        
    def connect(self):
        ethernet.sock.connect(self.fpgaHost, 4001)
        
    def disconnect(self):
        ethernet.sock.close()
        ethernet.sock = ethernet.Sock()
        
    def loadModel(self, filename=None):
        import ics.cobraOps.CobrasCalibrationProduct as cobraModel
        reload(cobraModel)

        if filename is None:
            filename = "/Users/cloomis/Sumire/PFS/git/ics_cobraOps/python/ics/cobraOps/usedXMLFile.xml"
        
        self.calibModel = cobraModel.CobrasCalibrationProduct(filename)
        
    def _freqToPeriod(self, freq):
        """ Convert frequency to 60ns ticks """        
        return int(round(16e3 / (freq)))

    def _periodToFreq(self, freq):
        """ Convert 60ns ticks to a frequency """
        return  (16e3 / per) if (per>=1) else 0

    def _mapCobraIndex(self, cobra):
        """ Convert our module + cobra to global cobra index for the calibration product. """
        
        return ((cobra.module - 1)*self.nCobrasPerModule + cobra.cobraNum-1)
    
    def reset(self, sectors=0x3f):
        err = func.RST(sectors)
        
    def powerCycle(self, sectors=0x3f):
        err = func.POW(sectors)

    def setFreq(self, cobras):
        for c in cobras:
            cobraIdx = self._mapCobraIndex(c)
            thetaPer = self._freqToPeriod(self.calibModel.motorFreq1[cobraIdx]/1000)
            phiPer = self._freqToPeriod(self.calibModel.motorFreq2[cobraIdx]/1000)

            # print(f'set {c.board},{c.cobra} to {thetaPer},{phiPer} {self.calibModel.motorFreq1[c.cobra]}')
            c.p = func.SetParams(p0=thetaPer, p1=phiPer, en=(True, True))
        err = func.SET(cobras)
        
    def moveAllSteps(self, cobras, steps, dirs):
        allSteps = [steps]*len(cobras)
        allDirs = [dirs]*len(cobras)
        
        self.moveSteps(cobras, allSteps, allDirs)
        
    def moveSteps(self, cobras, steps, dirs):
        
        if len(cobras) != len(steps):
            raise RuntimeError("number of steps must match number of cobras")
        if len(cobras) != len(dirs):
            raise RuntimeError("number of directions must match number of cobras")

        model = self.calibModel
        
        for c_i, c in enumerate(cobras):
            steps1 = steps[c_i]
            dirs1 = dirs[c_i]
            en = (steps1[0] != 0, steps1[1] != 0)
            cobraId = self._mapCobraIndex(c)
            
            if dirs1[0] == 'cw':
                ontime1 = model.motorOntimeFwd1[cobraId]
                offtime1 = model.motorOfftimeFwd1[cobraId]
            elif dirs1[0] == 'ccw':
                ontime1 = model.motorOntimeRev1[cobraId]
                offtime1 = model.motorOfftimeRev1[cobraId]
            else:
                raise ValueError(f'invalid direction: {dirs1[0]}')
                
            if dirs1[1] == 'cw':
                ontime2 = model.motorOntimeFwd2[cobraId]
                offtime2 = model.motorOfftimeFwd2[cobraId]
            elif dirs1[1] == 'ccw':
                ontime2 = model.motorOntimeRev2[cobraId]
                offtime2 = model.motorOfftimeRev2[cobraId]
            else:
                raise ValueError(f'invalid direction: {dirs1[1]}')
                
            c.p = func.RunParams(pu=(int(1000*ontime1), int(1000*ontime2)),
                                 st=(steps1),
                                 sl=(int(1000*offtime1), int(1000*offtime2)),
                                 en=en,
                                 dir=dirs1)
        err = func.RUN(cobras)
        
    @classmethod
    def allocateAllCobras(cls):
        return allocateCobraRange(range(1,cls.nModules))

    @classmethod
    def allocateCobraRange(cls, modules, cobraNums=None):
        """ Utility to allocate swaths of cobras:
    
        Args:
          modules (int array-like): a list of 1-indexed boards to allocate from.
          cobras  (int array-like): a list of 1-indexed cobras to allocate from.

        Return:
          cobras
        """
        cobras = []
    
        if np.isscalar(modules):
            modules = [modules]
        for m in modules:
            if m == 0:
                raise IndexError('module numbers are 1-indexed, grrr.')
            if cobraNums is None:
                _cobraNums = range(1,cls.nCobrasPerModule)
            else:
                _cobraNums = cobraNums
            
            for c in _cobraNums:
                if c == 0:
                    raise IndexError('cobra numbers are 1-indexed, grrr.')

                cobras.append(func.Cobra(m, c))
        
        return cobras

    @classmethod
    def allocateCobraList(cls, cobraIds):
        cobras = []
        for mc in cobraIds:
            m, c = mc
            if m == 0:
                raise IndexError('module numbers are 1-indexed, grrr.')
            if c == 0:
                raise IndexError('cobra numbers are 1-indexed, grrr.')

            cobras.append(func.Cobra(m, c))
        
        return cobras

def retractPhi(pfi, cobras, nsteps=100):
    steps = [(0,nsteps)]*len(cobras)
    dirs = [('ccw','ccw')]*len(cobras)
    pfi.moveSteps(cobras, steps, dirs)
    
def testRuns(cobras):
    for c in cobras:
        c.p = func.RunParams(pu=(70,70),
                             st=(10,10),
                             en=(True, True),
                             dir=('cw', 'cw'))

In [10]:
testCobras = "moduleGroups"

if testCobras == "allCobras":
    cobras = PFI.allocateCobraRange(range(1,43))
elif testCobras == "oneModule":
    cobras = PFI.allocateCobraRange(range(1,2))
elif testCobras == "oneCobra":
    cobras = PFI.allocateCobraList([(1,1)])    
elif testCobras == "moduleGroups":
    moduleCobras = {}
    for group in 1,2,3:
        cm = list(range(group,58,3))
        mod = list([1]*len(cm))
    
        moduleCobras[group] = PFI.allocateCobraList(list(zip(mod,cm)))
    cobras = moduleCobras[1]
else:
    raise RuntimeError()

In [11]:
print([f"{c.module},{c.cobraNum}" for c in cobras])

['1,1', '1,4', '1,7', '1,10', '1,13', '1,16', '1,19', '1,22', '1,25', '1,28', '1,31', '1,34', '1,37', '1,40', '1,43', '1,46', '1,49', '1,52', '1,55']


In [12]:
pfi = PFI()

In [13]:
pfi.setFreq(moduleCobras[1])

In [14]:
%pdb on
retractPhi(pfi, cobras)
%pdb off

Automatic pdb calling has been turned ON
Automatic pdb calling has been turned OFF


In [15]:
pfi.moveAllSteps(cobras, (5,5), dirs=('cw','cw'))

In [17]:
pfi.powerCycle()

In [12]:
print(pfi.calibModel.motorFreq1, pfi.calibModel.motorFreq2)
print(pfi.calibModel.motorOntimeFwd1, pfi.calibModel.motorOntimeFwd2)

[ 63241.  63241.  63745.  63492.  63241.  63492.  62500.  63492.  64000.
  63492.  62992.  63241.  62992.  63492.  64000.  63492.  63492.  62992.
  63241.  64000.  63745.  62992.  63492.  64000.  63241.  62500.  63241.
  63241.  63492.  63492.  63745.  63745.  63745.  63241.  63241.  64000.
  62745.  64000.  62745.  62992.  63492.  62500.  63241.  62256.  64000.
  63492.  64000.  62992.  63492.  64516.  64000.  64257.  64516.  63745.
  64000.  64777.  63492.] [ 108108.  104575.  103896.  104575.  105263.  105960.  105960.  105263.
  104575.  105960.  107382.  105263.  105960.  105263.  105960.  103896.
  105263.  106666.  105263.  105960.  105960.  104575.  104575.  105263.
  105960.  106666.  104575.  108108.  105960.  105263.  108843.  106666.
  105263.  105263.  106666.  105263.  105960.  105263.  104575.  106666.
  105960.  105263.  105960.  104575.  105960.  105263.  105960.  105263.
  106666.  106666.  105960.  105960.  104575.  104575.  103896.  107382.
  103896.]
[ 0.04572   0.

In [13]:
pfi.reset()

NameError: name 'sec_rst' is not defined